In [ ]:
from scipy.optimize import bisect
import numpy as np
import scipy as sc
from scipy.optimize import fsolve
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import scipy.constants as cnst
%matplotlib inline

'''

Outline:

Step 1: Define the four differnetial equations of the Hodgkin-Huxley Model:
- This includes:
  1) dV/dt = (Iext - Ina - Ik - Il)/Cm, which is the rate of change of the membrane potential
  Iext is the externa current through the neuron, which can vary, so we need to obtain data to solve this.
  Ina is the current passing through the sodium-voltage gate
  Ik is the current passing through the potassium-voltage gate
  Il is the leak current.
  Cm is the membrane capacitance.
  2) dndt = an*(1-n) - bn*n, which is the the Potassium activation gating variable
  'an' is the alpha value for the potassium activation
  'bn' is the beta value for the potasium activation
  n is the varible that describes the potassium activtion
  3) dmdt = am*(1-m) - bm*m, which is the the Sodium activation gating variable
  'am' is the alpha value for the sodium activation
  'bm' is the beta value for the sodium activation
  m is the varible that describes the sodium activtion
  4) dhdt = ah*(1-h) - bh*h, which is the the Sodium inactivation gating variable
  'ah' is the alpha value for the sodium inactivation
  'bh' is the beta value for the sodium inactivation
  h is the varible that describes the sodium inactivtion
Step 2: Numerically solve for these equations using odeint from scipy:
- This includes:
    1) Integrating the equations with odeint, with respect to time.
    2) Setting initial values for V (the resting potential V = -65 eV), the gating variables n, m and h.
    3) Setting a time step and a time duration (0 to 50 milliseconds)
Step 3: Simulate the Action potentials:
    1) This means we can apply a stimulus like injection or pain to initiate the action potential
    2) vary the magnitudes of the parameters and see how that affects initiation and propagation and the amplitude of our waves.
Step 4: Plot our graphs:
    1) Plotting the change in membrane potential with respect to time.
    2) Plotting each of the gated variables with respect to time.
    3) Identify the minimum stimuli require to initiate an action potential.
    
'''

def alpha_rtconsts(V): #Defines the alpha values for n, m, and h
    num_n = 10 - V
    num_m = 25 - V
    
    ah = 0.07*np.exp(-V/20)
    am = 0.1*(num_m)/(np.exp(num_m/10)-1)
    an = 0.01*(num_n)/(np.exp(num_n/10)-1)

    return ah, am, an

def beta_rtconsts(V): #Defines the beta values for n, m, and h, 
    num = 30-V
    
    bh = 1/(np.exp(num/10)+1)
    bm = 4*np.exp(-V/18)
    bn  = 0.125*np.exp(-V/80)

    return bh, bm, bn

def hod_hux(V, n, m, h, Iext): #This defines all the Hodgkin-huxley equations
    
    #The following values are determined from previous experimental data.
    gNa = 120 #mS/cm^2 (maximal Sodium conductance)
    gK = 36 #mS/cm^2 (maximal Potassium conductance)
    gL = 0.3 #mS/cm^2 (Leak conductance)
    ENa = 50 #mV (sodium equilibrium potential)
    EK = -77 #mV (potassium equilibrium potential)
    EL = -54.4 #mV (leak equilibrium potential)
    Cm = 1 #mu*F/cm^2 (Membrane capacitance)

    ah, am, an = alpha_rtconsts(V) 
    bh, bm, bn = beta_rtconsts(V)

    Ina = gNa*(m**3)*h*(V - ENa)
    Ik = gK*(n**4)*(V - EK)
    Il = gL*(V - EL)

    dVdt = (Iext - Ina - Ik - Il)/Cm #Defines the function for membrane potential w/ respect to time (dV_m/dt).

    dndt = an*(1-n) - bn*n #Defines the Potassium activation gating variable (dn/dt)
    dmdt = am*(1-m) - bm*m #Defines the Sodium activation gating variable (dm/dt)
    dhdt = ah*(1-h) - bh*h #Defines the Sodium inactivation gating variable (dh/dt)
    
    return dVdt, dndt, dmdt, dhdt
    
def m_vals(am, bm): #defines the value of m (sodium activation)
    return am/(am + bm)

def n_vals(an, bn): #defines the value of n (for potassium)
    return an/(an + bn)

def h_vals(ah, bh): #defines the value of h (Sodium inactivation)
    return ah/(ah + bh)

#the returned values are alpha, beta for h, m, and n
ah, am, an = alpha_rtconsts( , , )
bh, bm, bn = beta_rtconsts( , , )

#Input alpha and beta values to get the values for m, n, and h.
m = m_vals(am, bm)
n = n_vals(an, bn)
h = h_vals(ah, bh)

#Input alpha, beta and n,m,h values
#input the potential data and external current